In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 56.1 MB/s eta 0:00:00


In [ ]:


from transformers import pipeline

# Load the GPT-2 model for text generation
generator = pipeline('text2text-generation', model='gpt2')

# Input text for summarization
text = '''Reason: WEAKNESS MEDICAL CONDITION: 58 year old man with apml with shortness of breath. REASON FOR THIS EXAMINATION: assess for effusions infiltrate cause for hct drop dah FINAL REPORT INDICATION: 58yearold with APML and shortness of breath. Evaluate for effusions and infiltrates or cause for hematocrit drop. COMPARISON: CT of the chest . SINGLE AP UPRIGHT BEDSIDE CHEST RADIOGRAPH: Compared to the CT and chest radiograph from one day prior there has been significant interval improvement of the right upper lobe opacity which may have been due to aspiration given its marked improvement. There is a new right infrahilar opacity which may represent pneumonia or hemorrhage. There is perihilar haze and indistinctness of the vasculature consistent with mild pulmonary edema. The cardiomediastinal silhouette is normal. IMPRESSION: 1. Improved right upper lobe opacity with new right perihilar opacity. This new opacity may represent pneumonia or hemorrhage given the patients hematocrit drop. 2. Mild fluid overload. '''
num_lines = len(text.split('.'))
# Set the parameters for summary generation
max_length = len(text)//3 # Generate a summary that is one-third of the length of the input text
num_sentences = num_lines//2 # Generate a summary with two sentences

prefix = 'Summarize the medical report: ' + text
summary = generator(prefix, max_length=max_length, num_return_sequences=2, num_beams=5, no_repeat_ngram_size=2, length_penalty=2.0)[0]['generated_text'].replace(prefix, '').strip()





The model 'GPT2LMHeadModel' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
summary_sentences = summary.split('.')

In [ ]:
# Print the summary
print('. '.join(summary_sentences) + '.')

_______________________________________________________________________________

The following is an excerpt from an article published in the New England Journal of Medicine (NEJM) entitled "Pneumonia and Hematopoietic Injuries: A Case Report. " The NEJM article was written by Dr.  Michael J.  Schulman, MD, a professor of medicine at the University of North Carolina at Chapel Hill, and is available at http://www. ncbi. nlm. nih. gov/pub.


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas(desc='My bar!')
import numpy as np
import time
import matplotlib.pyplot as plt 

In [ ]:
df = pd.read_csv('cleaned_radiology.csv')
df  = df.drop('Unnamed: 0', axis=1)
print(df.columns)

Index(['icustay_id', 'hr', 'category', 'description', 'text', 'mortality',
       'cleaned_text'],
      dtype='object')


In [ ]:
df.dropna(subset=['category', 'description'], how='all', inplace=True)

In [ ]:
df.sample(5)

,icustay_id,hr,category,description,text,mortality,cleaned_text
2631,253146,9,Radiology,CT HEAD W/O CONTRAST,"[**Last Name (LF) **],[**First Name3 (LF) 817*...",0,Reason: INTRACRANIAL HEMORRHAGE MEDICAL CONDIT...
3829,277113,4,Radiology,CT HEAD W/O CONTRAST,[**2156-9-5**] 10:38 AM\n CT HEAD W/O CONTRAST...,1,Reason: INTRACRANIAL HEMORRHAGE MEDICAL CONDIT...
3668,273876,3,Radiology,CHEST (SINGLE VIEW),[**2122-9-18**] 2:34 AM\n CHEST (SINGLE VIEW) ...,0,Reason: INTRACRANIAL HEMORRHAGE MEDICAL CONDIT...
4741,297205,13,Radiology,CT HEAD W/O CONTRAST,[**2193-3-29**] 11:00 AM\n CT HEAD W/O CONTRAS...,0,Reason: STROKETIA MEDICAL CONDITION: 88 year o...
4643,295469,17,Radiology,CT HEAD W/O CONTRAST,[**2141-11-12**] 8:44 PM\n CT HEAD W/O CONTRAS...,0,Reason: SUBARACHNOID HEMORRHAGE MEDICAL CONDIT...


In [ ]:
# 'text' column is an object but not all strings
df['text'] = df['text'].astype(str)

In [ ]:
report = df.loc[1050, 'text']
print(report)

[**2183-4-20**] 9:05 PM
 CHEST (PORTABLE AP)                                             Clip # [**Clip Number (Radiology) 50966**]
 Reason: assess for effusions, infiltrate, cause for hct drop (dah)
 Admitting Diagnosis: WEAKNESS
 ______________________________________________________________________________
 [**Hospital 4**] MEDICAL CONDITION:
  58 year old man with apml with shortness of breath.
 REASON FOR THIS EXAMINATION:
  assess for effusions, infiltrate, cause for hct drop (dah)
 ______________________________________________________________________________
                                 FINAL REPORT
 INDICATION:  58-year-old with APML and shortness of breath.  Evaluate for
 effusions and infiltrates, or cause for hematocrit drop.

 COMPARISON:  CT of the chest, [**2183-4-19**].

 SINGLE AP UPRIGHT BEDSIDE CHEST RADIOGRAPH:  Compared to the CT and chest
 radiograph from one day prior, there has been significant interval improvement
 of the right upper lobe opacity, which may

In [ ]:
len(report.split())

181

In [ ]:
df.shape

(4882, 7)

# Cleaning step 1 - removing non-informative text

In [ ]:
import re

def custom_cleaner(text):
  text = re.sub('\s+', ' ', text).strip()
  text = re.sub(r'\n\.', ' ', text)
  # regular expression to match all substrings between square brackets
  pattern = r"\[.*?\]"
    
  # substitute all matches with an empty string
  text = re.sub(pattern, "", text)

  # Remove time stamps
  text = re.sub(r'\d+:\d+\s*(?:AM|PM)', '', text)

  # Remove all characters except alphabets, full stop, and colon
  text = re.sub(r'[^a-zA-Z0-9.:\s]', '', text)

  colon_index = text.find(':')
  if colon_index != -1:
      text = text[0:colon_index+1] + text[colon_index+1:].split(':', 1)[1]

  words = text.split()
  output = ""
  colon_found = False

  for word in words:
      if ":" in word:
          output += word + " "
          colon_found = True
      elif colon_found:
          output += word + " "
  if output != "":
    text = output
  return text


In [ ]:
clean_report = custom_cleaner(report)

In [ ]:
len(clean_report.split())

154

In [ ]:
print(clean_report)

Reason: WEAKNESS MEDICAL CONDITION: 58 year old man with apml with shortness of breath. REASON FOR THIS EXAMINATION: assess for effusions infiltrate cause for hct drop dah FINAL REPORT INDICATION: 58yearold with APML and shortness of breath. Evaluate for effusions and infiltrates or cause for hematocrit drop. COMPARISON: CT of the chest . SINGLE AP UPRIGHT BEDSIDE CHEST RADIOGRAPH: Compared to the CT and chest radiograph from one day prior there has been significant interval improvement of the right upper lobe opacity which may have been due to aspiration given its marked improvement. There is a new right infrahilar opacity which may represent pneumonia or hemorrhage. There is perihilar haze and indistinctness of the vasculature consistent with mild pulmonary edema. The cardiomediastinal silhouette is normal. IMPRESSION: 1. Improved right upper lobe opacity with new right perihilar opacity. This new opacity may represent pneumonia or hemorrhage given the patients hematocrit drop. 2. Mi

In [ ]:
print(type(report))

<class 'str'>


In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelWithLMHead.from_pretrained("gpt2")
# Add a new padding token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Define the text to be summarized
text = clean_report
# Tokenize the text
inputs = tokenizer.encode( text, return_tensors="pt", max_length=512, truncation=True)

# Generate the summary
summary_ids = model.generate(inputs, max_length=512, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the summary
print(summary)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Reason: CEREBRAL INFARCTION MEDICAL CONDITION: 74 year old man with cerebellar infarct hemorrhage 4th ventricle occlusion. REASON FOR THIS EXAMINATION: compare to outside hospital MRI. No contraindications for IV contrast FINAL REPORT HISTORY: Cerebellar infarction hemorrhage fourth ventricle occlusion. Compared to outside hospital MRI. COMPARISON: No previous studies in PACS. The outside hospital MRI is not available for comparison. TECHNIQUE: Noncontrast head CT. FINDINGS: There is a large subacute infarction in the right cerebellar hemisphere which extends to the vermis with associated compression of the fourth ventricle. The lateral and third ventricles are dilated. Evaluation of the lower posterior fossa for hemorrhage is limited by bone related artifact and motion artifact. Small amount of blood may be present within the right cerebellar infarction. There is a chronic infarction in the left cerebellar hemisphere. There are multiple small foci of low density in the subcortical and

In [ ]:
# Print the summary
print(summary)

summarize: Reason: CEREBRAL INFARCTION MEDICAL CONDITION: 74 year old man with cerebellar infarct hemorrhage 4th ventricle occlusion. REASON FOR THIS EXAMINATION: compare to outside hospital MRI. No contraindications for IVF or IVF medication.

Cerebellar Infarction

Cerebellar infarct hemorrhage is the most common cause of


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the pre-trained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2',padding_side='left')
tokenizer.pad_token = tokenizer.eos_token
# Set the maximum length of the input and summary
max_input_length = 512
max_summary_length = 50

def generate_gpt2_summary(text, model=model, tokenizer=tokenizer, max_input_length=max_input_length, max_summary_length=max_summary_length):
  # Tokenize the input data
  input_ids = tokenizer.encode("summarize: " +text, max_length=max_input_length,truncation=True, padding='max_length', return_tensors='pt')

  # Generate the summary
  summary_ids = model.generate(input_ids, num_beams=4, max_length=max_input_length+1, early_stopping=False)
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

  return summary


In [ ]:
long_text = "t is a long established fact that a reader will be distracted by the readable content of a page when looking at its layout. The point of using Lorem Ipsum is that it has a more-or-less normal distribution of letters, as opposed to using 'Content here, content here', making it look like readable English. Many desktop publishing packages and web page editors now use Lorem Ipsum as their default model text, and a search for 'lorem ipsum' will uncover many web sites still in their infancy. Various versions have evolved over the years, sometimes by accident, sometimes on purpose (injected humour and the like)."

In [ ]:
report_summary = generate_gpt2_summary(text=clean_report)
print(report_summary)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


summarize: Reason: CEREBRAL INFARCTION MEDICAL CONDITION: 74 year old man with cerebellar infarct hemorrhage 4th ventricle occlusion. REASON FOR THIS EXAMINATION: compare to outside hospital MRI. No contraindications for IV contrast FINAL REPORT HISTORY: Cerebellar infarction hemorrhage fourth ventricle occlusion. Compared to outside hospital MRI. COMPARISON: No previous studies in PACS. The outside hospital MRI is not available for comparison. TECHNIQUE: Noncontrast head CT. FINDINGS: There is a large subacute infarction in the right cerebellar hemisphere which extends to the vermis with associated compression of the fourth ventricle. The lateral and third ventricles are dilated. Evaluation of the lower posterior fossa for hemorrhage is limited by bone related artifact and motion artifact. Small amount of blood may be present within the right cerebellar infarction. There is a chronic infarction in the left cerebellar hemisphere. There are multiple small foci of low density in the subc

In [ ]:
print(len(long_text))

612


In [ ]:
s = "Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem"

In [ ]:
print(len(s))

451


In [ ]:
print(len(report_summary))

1186


In [ ]:
print(clean_report)

Reason: CEREBRAL INFARCTION MEDICAL CONDITION: 74 year old man with cerebellar infarct hemorrhage 4th ventricle occlusion. REASON FOR THIS EXAMINATION: compare to outside hospital MRI. No contraindications for IV contrast FINAL REPORT HISTORY: Cerebellar infarction hemorrhage fourth ventricle occlusion. Compared to outside hospital MRI. COMPARISON: No previous studies in PACS. The outside hospital MRI is not available for comparison. TECHNIQUE: Noncontrast head CT. FINDINGS: There is a large subacute infarction in the right cerebellar hemisphere which extends to the vermis with associated compression of the fourth ventricle. The lateral and third ventricles are dilated. Evaluation of the lower posterior fossa for hemorrhage is limited by bone related artifact and motion artifact. Small amount of blood may be present within the right cerebellar infarction. There is a chronic infarction in the left cerebellar hemisphere. There are multiple small foci of low density in the subcortical and

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Running the T5-base model on all cells

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
# Define the input data
input_text2 = clean_report


In [ ]:
# Tokenize the input data
input_ids = tokenizer.encode("summarize: " +input_text2, max_length=500,truncation=True, padding='max_length', return_tensors='pt')

# Generate the summary
summary_ids = model.generate(input_ids, num_beams=4, max_length=500, early_stopping=False)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the summary
print("Summary:", summary)

Summary: 74 year old man with cerebellar infarct hemorrhage 4th ventricle occlusion. compared to outside hospital MRI; no contraindications for IV contrast. small amount of blood may be present within the right cerebellar infarction.


In [ ]:
def sumarizer(text):
    # Tokenize the input data
    input_ids = tokenizer.encode("summarize: " +text, max_length=500,truncation=True, padding='max_length', return_tensors='pt')

    # Generate the summary
    summary_ids = model.generate(input_ids, num_beams=4, max_length=500, early_stopping=False)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Print the summary
    return summary


In [ ]:
tentative = sumarizer(df2.loc[5, 'cleaned_text'])

NameError: ignored

In [ ]:
len(tentative.split())

38

In [ ]:
df2['summary_T5'] = df2['cleaned_text'].progress_apply(sumarizer)

My bar!: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 4882/4882 [8:47:18<00:00,  6.48s/it]


In [ ]:
df2.head()

,Unnamed: 0,icustay_id,hr,category,description,text,mortality,cleaned_text,summary_T5
0,0,200039,12,Radiology,CT HEAD W/O CONTRAST,[**2121-12-26**] 4:56 PM\n CT HEAD W/O CONTRAS...,0,Reason: 69 year old woman sp decompressive sur...,69 year old woman sp decompressive surgery for...
1,1,200052,20,Radiology,CT HEAD W/O CONTRAST,[**2193-9-19**] 8:41 AM\n CT HEAD W/O CONTRAST...,0,Reason: CEREBELLER HEMORRHAGE MEDICAL CONDITIO...,57 year old woman with cerebellar hemorrhage. ...
2,2,200072,6,Radiology,CT HEAD W/O CONTRAST,[**2106-3-4**] 3:26 AM\n CT HEAD W/O CONTRAST ...,0,Reason: CEREBELLAR HEMORRHAGE MEDICAL CONDITIO...,MEDICAL CONDITION: 55 year old male with left ...
3,3,200072,8,Radiology,MR HEAD W & W/O CONTRAST,[**2106-3-4**] 5:26 AM\n MR HEAD W & W/O CONTR...,0,Reason: CEREBELLAR HEMORRHAGE Contrast: MAGNEV...,a 55 year old man with acute onset diplopia an...
4,4,200072,8,Radiology,MR HEAD W & W/O CONTRAST,"[**Last Name (LF) 477**],[**First Name3 (LF) 4...",0,Reason: CEREBELLAR HEMORRHAGE Contrast: MAGNEV...,man with acute onset diplopia and imbalance hy...


In [ ]:
# how many words in the text column
df2["Number_Words"] = df2["text"].apply(lambda n: len(n.split()))
df2.head()

,Unnamed: 0,icustay_id,hr,category,description,text,mortality,cleaned_text,summary_T5,Number_Words
0,0,200039,12,Radiology,CT HEAD W/O CONTRAST,[**2121-12-26**] 4:56 PM\n CT HEAD W/O CONTRAS...,0,Reason: 69 year old woman sp decompressive sur...,69 year old woman sp decompressive surgery for...,184
1,1,200052,20,Radiology,CT HEAD W/O CONTRAST,[**2193-9-19**] 8:41 AM\n CT HEAD W/O CONTRAST...,0,Reason: CEREBELLER HEMORRHAGE MEDICAL CONDITIO...,57 year old woman with cerebellar hemorrhage. ...,141
2,2,200072,6,Radiology,CT HEAD W/O CONTRAST,[**2106-3-4**] 3:26 AM\n CT HEAD W/O CONTRAST ...,0,Reason: CEREBELLAR HEMORRHAGE MEDICAL CONDITIO...,MEDICAL CONDITION: 55 year old male with left ...,202
3,3,200072,8,Radiology,MR HEAD W & W/O CONTRAST,[**2106-3-4**] 5:26 AM\n MR HEAD W & W/O CONTR...,0,Reason: CEREBELLAR HEMORRHAGE Contrast: MAGNEV...,a 55 year old man with acute onset diplopia an...,244
4,4,200072,8,Radiology,MR HEAD W & W/O CONTRAST,"[**Last Name (LF) 477**],[**First Name3 (LF) 4...",0,Reason: CEREBELLAR HEMORRHAGE Contrast: MAGNEV...,man with acute onset diplopia and imbalance hy...,96


In [ ]:
df2["Number_Words"] = df2["text"].apply(lambda n: len(n.split()))
df2["Number_Words_T5"] = df2["summary_T5"].apply(lambda n: len(n.split()))
df2.head(20)

,Unnamed: 0,icustay_id,hr,category,description,text,mortality,cleaned_text,summary_T5,Number_Words,Number_Words_T5
0,0,200039,12,Radiology,CT HEAD W/O CONTRAST,[**2121-12-26**] 4:56 PM\n CT HEAD W/O CONTRAS...,0,Reason: 69 year old woman sp decompressive sur...,69 year old woman sp decompressive surgery for...,184,22
1,1,200052,20,Radiology,CT HEAD W/O CONTRAST,[**2193-9-19**] 8:41 AM\n CT HEAD W/O CONTRAST...,0,Reason: CEREBELLER HEMORRHAGE MEDICAL CONDITIO...,57 year old woman with cerebellar hemorrhage. ...,141,21
2,2,200072,6,Radiology,CT HEAD W/O CONTRAST,[**2106-3-4**] 3:26 AM\n CT HEAD W/O CONTRAST ...,0,Reason: CEREBELLAR HEMORRHAGE MEDICAL CONDITIO...,MEDICAL CONDITION: 55 year old male with left ...,202,29
3,3,200072,8,Radiology,MR HEAD W & W/O CONTRAST,[**2106-3-4**] 5:26 AM\n MR HEAD W & W/O CONTR...,0,Reason: CEREBELLAR HEMORRHAGE Contrast: MAGNEV...,a 55 year old man with acute onset diplopia an...,244,40
4,4,200072,8,Radiology,MR HEAD W & W/O CONTRAST,"[**Last Name (LF) 477**],[**First Name3 (LF) 4...",0,Reason: CEREBELLAR HEMORRHAGE Contrast: MAGNEV...,man with acute onset diplopia and imbalance hy...,96,22
5,5,200072,22,Radiology,CHEST (PORTABLE AP),[**2106-3-25**] 1:18 PM\n CHEST (PORTABLE AP) ...,0,Reason: CEREBELLAR HEMORRHAGE MEDICAL CONDITIO...,55 year old man sp CABG eval for pneumothorax ...,142,38
6,6,200103,15,Radiology,SEL CATH 3RD ORDER THOR,[**2201-5-18**] 10:55 AM\n CAROT/CEREB [**Hosp...,0,Reason: SUBARACHNOID HEMORRHAGE Contrast: OPTI...,58-year-old man with sAH had a 19gauge single ...,665,62
7,7,200131,16,Radiology,CT HEAD W/O CONTRAST,[**2176-10-31**] 5:25 AM\n CT HEAD W/O CONTRAS...,0,Reason: STROKETELEMETRYTRANSIENT ISCHEMIC ATTA...,pontine hemorrhage appears slightly larger in ...,164,20
8,8,200131,21,Radiology,CHEST (PORTABLE AP),[**2176-10-31**] 10:07 AM\n CHEST (PORTABLE AP...,0,Reason: STROKETELEMETRYTRANSIENT ISCHEMIC ATTA...,69 year old woman with pontine hemorrhage and ...,168,27
9,9,200131,21,Radiology,CHEST (PORTABLE AP),"[**Last Name (LF) **],[**First Name8 (NamePatt...",0,Reason: STROKETELEMETRYTRANSIENT ISCHEMIC ATTA...,69 year old woman with pontine hemorrhage feve...,79,25


In [ ]:
df2.to_csv("t5results.csv", index=False)